In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions
from pyspark.sql import Row

In [2]:
def parseInput(line):
    fields = line.split('|')
    return Row(user_id = int(fields[0]), age = int(fields[1]), gender = fields[2], occupation = fields[3], zip = fields[4])

In [3]:
spark = SparkSession.builder.appName("MongoDBIntegration").getOrCreate()

In [4]:
lines = spark.sparkContext.textFile("/home/mak/Downloads/hadoop/ml-100k/u.user")

In [5]:
users = lines.map(parseInput)

In [6]:
usersDataset = spark.createDataFrame(users)

In [7]:
usersDataset.write.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/movielens.users").mode('append').save()

In [8]:
readUsers = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/movielens.users").load()

In [9]:
readUsers.createOrReplaceTempView("users")

In [10]:
sqlDF = spark.sql("SELECT * FROM users WHERE age < 20")

In [11]:
sqlDF.show()

+--------------------+---+------+-------------+-------+-----+
|                 _id|age|gender|   occupation|user_id|  zip|
+--------------------+---+------+-------------+-------+-----+
|[5bc89fd9c047fd0a...|  7|     M|      student|     30|55436|
|[5bc89fd9c047fd0a...| 19|     F|      student|     36|93117|
|[5bc89fd9c047fd0a...| 18|     F|      student|     52|55105|
|[5bc89fd9c047fd0a...| 16|     M|         none|     57|84010|
|[5bc89fd9c047fd0a...| 17|     M|      student|     67|60402|
|[5bc89fd9c047fd0a...| 19|     M|      student|     68|22904|
|[5bc89fd9c047fd0a...| 15|     M|      student|    101|05146|
|[5bc89fd9c047fd0a...| 19|     M|      student|    110|77840|
|[5bc89fd9c047fd0a...| 13|     M|        other|    142|48118|
|[5bc89fd9c047fd0a...| 15|     M|entertainment|    179|20755|
|[5bc89fd9c047fd0a...| 14|     F|      student|    206|53115|
|[5bc89fd9c047fd0a...| 19|     M|      student|    221|20685|
|[5bc89fd9c047fd0a...| 19|     F|      student|    223|47906|
|[5bc89f

In [12]:
spark.stop()